# NLP-Based Spelling Autocorrector

This notebook implements a complete spelling autocorrection system using:
- Text corpus analysis
- Word frequency probabilities
- Edit distance operations
- Candidate filtering and ranking

## 1. Setup and Imports

In [ ]:
import re
from collections import Counter
import os
import glob

## 2. Load Dataset

In [ ]:
def load_corpus(directory_path='/mnt/project'):
    """
    Load all text files from the specified directory.
    """
    text = ""
    
    # Get all .txt files
    file_pattern = os.path.join(directory_path, '*.txt')
    files = glob.glob(file_pattern)
    
    print(f"Found {len(files)} text files")
    
    for filepath in files:
        try:
            with open(filepath, 'r', encoding='utf-8') as f:
                text += f.read() + " "
        except Exception as e:
            print(f"Error reading {filepath}: {e}")
    
    print(f"Total characters loaded: {len(text):,}")
    return text

# Load the corpus
corpus_text = load_corpus()

## 3. Preprocessing

In [ ]:
def preprocess_text(text):
    """
    Convert to lowercase and extract alphabetic words.
    """
    # Lowercase
    text = text.lower()
    
    # Extract words (alphabetic only)
    words = re.findall(r'[a-z]+', text)
    
    return words

# Process the corpus
words = preprocess_text(corpus_text)
print(f"Total words extracted: {len(words):,}")
print(f"Sample words: {words[:20]}")

## 4. Build Vocabulary and Calculate Frequencies

In [ ]:
def build_vocabulary(words):
    """
    Create word frequency counter and vocabulary set.
    """
    word_counts = Counter(words)
    vocabulary = set(words)
    total_words = len(words)
    
    return word_counts, vocabulary, total_words

# Build vocabulary
word_counts, vocabulary, total_words = build_vocabulary(words)

print(f"Unique words in vocabulary: {len(vocabulary):,}")
print(f"\nTop 20 most common words:")
for word, count in word_counts.most_common(20):
    print(f"  {word}: {count:,}")

In [ ]:
def word_probability(word, word_counts, total_words):
    """
    Calculate probability of a word: P(word) = count(word) / total_words
    """
    return word_counts.get(word, 0) / total_words

# Test probability calculation
test_words = ['the', 'and', 'computer', 'xyzabc']
print("Word probabilities:")
for word in test_words:
    prob = word_probability(word, word_counts, total_words)
    print(f"  P('{word}') = {prob:.8f}")

## 5. Edit Operations

In [ ]:
def deletion_edits(word):
    """
    Generate all possible words by deleting one character.
    Example: 'test' -> ['est', 'tst', 'tet', 'tes']
    """
    return [word[:i] + word[i+1:] for i in range(len(word))]

def insertion_edits(word):
    """
    Generate all possible words by inserting one character.
    """
    letters = 'abcdefghijklmnopqrstuvwxyz'
    return [word[:i] + c + word[i:] for i in range(len(word) + 1) for c in letters]

def substitution_edits(word):
    """
    Generate all possible words by substituting one character.
    """
    letters = 'abcdefghijklmnopqrstuvwxyz'
    return [word[:i] + c + word[i+1:] for i in range(len(word)) for c in letters if c != word[i]]

def transposition_edits(word):
    """
    Generate all possible words by swapping adjacent characters.
    Example: 'test' -> ['etst', 'tset', 'tets']
    """
    return [word[:i] + word[i+1] + word[i] + word[i+2:] for i in range(len(word) - 1)]

# Test edit operations
test_word = "test"
print(f"Edit operations on '{test_word}':")
print(f"  Deletions (first 5): {deletion_edits(test_word)[:5]}")
print(f"  Insertions (first 5): {insertion_edits(test_word)[:5]}")
print(f"  Substitutions (first 5): {substitution_edits(test_word)[:5]}")
print(f"  Transpositions: {transposition_edits(test_word)}")

## 6. Generate Edit-1 Candidates

In [ ]:
def edit_one_candidates(word):
    """
    Generate all possible edit-distance-1 candidates.
    """
    candidates = set()
    
    candidates.update(deletion_edits(word))
    candidates.update(insertion_edits(word))
    candidates.update(substitution_edits(word))
    candidates.update(transposition_edits(word))
    
    return candidates

# Test candidate generation
test_word = "speling"
candidates = edit_one_candidates(test_word)
print(f"Generated {len(candidates)} edit-1 candidates for '{test_word}'")
print(f"Sample candidates: {list(candidates)[:10]}")

## 7. Filter and Rank Candidates

In [ ]:
def filter_candidates(candidates, vocabulary):
    """
    Keep only candidates that exist in the vocabulary.
    """
    return [word for word in candidates if word in vocabulary]

def rank_candidates(candidates, word_counts, total_words):
    """
    Rank candidates by probability and return the best one.
    """
    if not candidates:
        return None
    
    # Calculate probabilities and sort
    candidates_with_prob = [(word, word_probability(word, word_counts, total_words)) 
                            for word in candidates]
    candidates_with_prob.sort(key=lambda x: x[1], reverse=True)
    
    return candidates_with_prob[0][0]

# Test filtering and ranking
test_word = "speling"
all_candidates = edit_one_candidates(test_word)
valid_candidates = filter_candidates(all_candidates, vocabulary)
print(f"Valid candidates for '{test_word}': {valid_candidates[:10]}")

best = rank_candidates(valid_candidates, word_counts, total_words)
print(f"Best correction: '{best}'")

## 8. Edit-2 Candidates (Optional)

In [ ]:
def edit_two_candidates(word):
    """
    Generate edit-distance-2 candidates (edits of edits).
    Warning: This can generate a very large set.
    """
    edit1 = edit_one_candidates(word)
    edit2 = set()
    
    for w in edit1:
        edit2.update(edit_one_candidates(w))
    
    return edit2

# Test (on a short word to avoid excessive computation)
test_word = "helo"
edit2 = edit_two_candidates(test_word)
print(f"Generated {len(edit2)} edit-2 candidates for '{test_word}'")

## 9. Complete Autocorrect Function

In [ ]:
def autocorrect(word, vocabulary, word_counts, total_words, use_edit2=True):
    """
    Main autocorrect function.
    
    Args:
        word: Input word to correct
        vocabulary: Set of valid words
        word_counts: Counter object with word frequencies
        total_words: Total word count
        use_edit2: Whether to use edit-distance-2 if edit-1 fails
    
    Returns:
        Corrected word (or original if no correction found)
    """
    # Preprocess input
    word = word.lower().strip()
    
    # If word is already correct, return it
    if word in vocabulary:
        return word
    
    # Try edit-distance-1
    candidates = edit_one_candidates(word)
    valid_candidates = filter_candidates(candidates, vocabulary)
    
    if valid_candidates:
        return rank_candidates(valid_candidates, word_counts, total_words)
    
    # Try edit-distance-2 if enabled
    if use_edit2:
        candidates_2 = edit_two_candidates(word)
        valid_candidates_2 = filter_candidates(candidates_2, vocabulary)
        
        if valid_candidates_2:
            return rank_candidates(valid_candidates_2, word_counts, total_words)
    
    # No correction found, return original
    return word

## 10. Testing with Examples

In [ ]:
# Test cases: common misspellings
test_cases = [
    "speling",
    "korrect",
    "computr",
    "langauge",
    "progam",
    "recieve",
    "teh",
    "thier",
    "occured",
    "definately",
    "seperate",
    "goverment",
    "fredom",
    "consitution",
    "amercan"
]

print("Autocorrect Results:")
print("=" * 50)
for word in test_cases:
    corrected = autocorrect(word, vocabulary, word_counts, total_words)
    print(f"{word:20} -> {corrected}")

In [ ]:
# Test with correctly spelled words (should return unchanged)
correct_words = ["government", "freedom", "constitution", "american", "spelling"]

print("\nCorrectly Spelled Words (should remain unchanged):")
print("=" * 50)
for word in correct_words:
    corrected = autocorrect(word, vocabulary, word_counts, total_words)
    status = "✓" if word == corrected else "✗"
    print(f"{status} {word:20} -> {corrected}")

## 11. Interactive User Input

In [ ]:
def autocorrect_sentence(sentence, vocabulary, word_counts, total_words):
    """
    Autocorrect all words in a sentence.
    """
    words = re.findall(r'[a-zA-Z]+', sentence)
    corrected_words = [autocorrect(word, vocabulary, word_counts, total_words) 
                       for word in words]
    return ' '.join(corrected_words)

# Example usage
test_sentences = [
    "Ths is a tst sentance.",
    "The goverment maks importnt decisons.",
    "Fredom and democrcy are importnt valus."
]

print("Sentence Autocorrection:")
print("=" * 70)
for sentence in test_sentences:
    corrected = autocorrect_sentence(sentence, vocabulary, word_counts, total_words)
    print(f"Original:  {sentence}")
    print(f"Corrected: {corrected}")
    print()

In [ ]:
# Interactive input cell
# Run this cell and type your text below

user_input = input("Enter a word or sentence to autocorrect: ")
corrected = autocorrect_sentence(user_input, vocabulary, word_counts, total_words)

print(f"\nOriginal:  {user_input}")
print(f"Corrected: {corrected}")

## 12. Performance Analysis

In [ ]:
def analyze_correction(word, vocabulary, word_counts, total_words):
    """
    Detailed analysis of the correction process.
    """
    print(f"\nAnalyzing: '{word}'")
    print("=" * 50)
    
    # Check if already correct
    if word in vocabulary:
        print(f"'{word}' is already in vocabulary (correctly spelled)")
        return word
    
    # Generate and filter edit-1 candidates
    edit1 = edit_one_candidates(word)
    valid1 = filter_candidates(edit1, vocabulary)
    print(f"Edit-1 candidates: {len(edit1)} generated, {len(valid1)} valid")
    
    if valid1:
        # Show top 5 candidates with probabilities
        candidates_prob = [(w, word_probability(w, word_counts, total_words)) 
                          for w in valid1]
        candidates_prob.sort(key=lambda x: x[1], reverse=True)
        
        print("\nTop candidates:")
        for w, p in candidates_prob[:5]:
            print(f"  {w:15} P={p:.8f}")
        
        best = candidates_prob[0][0]
        print(f"\nBest correction: '{best}'")
        return best
    
    print("No valid edit-1 candidates found.")
    return word

# Analyze some examples
analyze_correction("speling", vocabulary, word_counts, total_words)
analyze_correction("goverment", vocabulary, word_counts, total_words)
analyze_correction("teh", vocabulary, word_counts, total_words)

## Summary

This autocorrect system:
- Loads a text corpus and builds a vocabulary
- Calculates word probabilities based on frequency
- Generates correction candidates using edit operations
- Filters candidates to valid words
- Selects the most probable correction

The system works well for common typos within 1-2 edits of the correct word.